# Decorators
We've seen several examples of the use of decorators (`@something` before a function definition) to change the behaviour of functions. My aim this week is to look at how these work and how they can be created.

There are several good introductions to decorators on the web covering more or less the material below. I've used the following:

* [Python Decorators](https://pythonconquerstheuniverse.wordpress.com/2012/04/29/python-decorators/) - a short and clear introduction.
* [A guide to Python's function decorators](http://thecodeship.com/patterns/guide-to-python-function-decorators/) - Ayman Farhat's introduction with a little more detail on the more advanced stuff.
* [Understanding Python Decorators in 12 Easy Steps!](http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/) - Simeon Franklin's step by step guide
* [Python Decorators - A practical application in science](http://tooblippe.github.io/insightstack-blog/2014/05/05/decorators/) - a nice example by Nelis Willers

For reference, decorators were introduced in 2003 into Python 2.4. See [PEP 318 -- Decorators for Functions and Methods](https://www.python.org/dev/peps/pep-0318/) for details.

### Motivating example
The numba documentation gives an example application where the summation of values of a numpy array is speeded up by adding `@jit`. But what does this `@` syntax do? How can we use it?

In [1]:
from numpy import arange

def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = arange(81000000).reshape(9000,9000)
print(sum2d(a))

3280499959500000.0


In [2]:
from numba import jit
from numpy import arange

@jit
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = arange(81000000).reshape(9000,9000)
print(sum2d(a))

3280499959500000.0


### Recap of functions in python
We use `def foo():` to define functions. Functions are "first class objects" - we can assign them to variables, pass them into other functions etc. so `foo` is a reference to the function  `foo()` evaluates the function and returns the result.

In [3]:
def say_hello(name):
    """Say hello to somebody"""
    mesg = "Hello, " + name
    return mesg

In [4]:
print(say_hello)

<function say_hello at 0x120117b00>


In [5]:
print(say_hello('Andrew'))

Hello, Andrew


In [6]:
hello_two = say_hello

In [7]:
print(hello_two('Andrew'))

Hello, Andrew


### Functions as arguments and return values
This means we can define a function that takes a function as an argument and returns a function as its result. We can use this to make it look like the behaviour of a function changes.  

In [8]:
def make_bold(function):
    def bold_function(string):
        return '<b>' + function(string) + '</b>'
    return bold_function

In [9]:
say_hello = make_bold(say_hello)

In [10]:
print(say_hello('Andrew'))

<b>Hello, Andrew</b>


In [11]:
def make_name_italic(function):
    def italic_function(string):
        string = '<i>' + string + '</i>'
        return(function(string))
    return italic_function

In [12]:
say_hello = make_name_italic(say_hello)

In [13]:
print(say_hello('Andrew'))

<b>Hello, <i>Andrew</i></b>


### Decorator lines
A shortcut for `new_function = decorator(old_func)` is provided by the `@` syntax.

In [14]:
@make_name_italic
@make_bold
def say_goodbye(name):
    """Say goodbye to somebody"""
    msg = 'Goodbye ' + name
    return msg

In [15]:
print(say_goodbye('Andrew'))

<b>Goodbye <i>Andrew</i></b>


### Decorator arguments
We may want to say:

    @add_tag(`b`)
    def say_goodbye(name):
       ...
       
but how do we do this?

In [16]:
def add_tag(tag):
    def add_tag_decorator(function):
        def tagged_function(name):
            msg = '<' + tag + '>' + function(name) + '</' + tag + '>'
            return msg
        return tagged_function
    return add_tag_decorator

In [17]:
@add_tag('b')
@add_tag('i')
def say_hi(name):
    """Say hi to somebody"""
    msg = 'Hi ' + name
    return msg


In [18]:
print(say_hi('Andrew'))

<b><i>Hi Andrew</i></b>


### Keeping the documentation (and other information)
Consider the following. Adding the decorators has made debugging more dificult.

In [19]:
print(say_hi.__doc__)

None


In [20]:
print(say_hi.__name__)

tagged_function


In [21]:
import functools
def add_tag(tag):
    def add_tag_decorator(function):
        @functools.wraps(function)
        def tagged_function(name):
            msg = '<' + tag + '>' + function(name) + '</' + tag + '>'
            return msg
        return tagged_function
    return add_tag_decorator

In [22]:
@add_tag('b')
@add_tag('i')
def say_hi(name):
    """Say hi to somebody"""
    msg = 'Hi ' + name
    return msg

In [23]:
say_hi.__doc__

'Say hi to somebody'

In [24]:
say_hi.__name__

'say_hi'

### More advanced decorators...

In [25]:
def count_calls(func):
    @functools.wraps(func)
    def wrapped_function(*args, **kwargs):
        wrapped_function.calls = wrapped_function.calls + 1
        return func(*args, **kwargs)
    wrapped_function.calls = 0
    return wrapped_function

In [26]:
@count_calls
def say_hi(name):
    """Say hi to somebody"""
    msg = 'Hi ' + name
    return msg

In [27]:
say_hi('andrew')

'Hi andrew'

In [28]:
say_hi('andrew')

'Hi andrew'

In [29]:
say_hi('andrew')

'Hi andrew'

In [30]:
say_hi.calls

3